In [2]:
from PIL import Image
import cv2 as cv
import numpy as np
import os
from tqdm import tqdm
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow import keras
from keras.optimizers import Adam
import tensorflow.keras.layers as k
import pickle

#### Building the dataset

In [3]:
# parameters
dataset = []
labels = []
eligible_type = ['jpg','png']
height = 70
width = 70

In [4]:
# loading closed eye images
eye_close_folder = os.listdir('./Dataset/Closed_Eyes/')

for img_id, each_image in enumerate(tqdm(eye_close_folder)):
    if each_image.split('.')[1] in eligible_type:
        image = cv.imread('./Dataset/Closed_Eyes/'+each_image)
        image = Image.fromarray(image,'RGB')
        image = image.resize((height,width))
        image = np.array(image)

        dataset.append(image)
        labels.append(0)

  0%|          | 0/2726 [00:00<?, ?it/s]

100%|██████████| 2726/2726 [00:07<00:00, 378.11it/s]


In [5]:
# loading open eye images
eye_open_folder = os.listdir('./Dataset/Open_Eyes/')

for img_id, each_image in enumerate(tqdm(eye_open_folder)):
    if each_image.split('.')[1] in eligible_type:
        image = cv.imread('./Dataset/Open_Eyes/'+each_image)
        image = Image.fromarray(image,'RGB')
        image = image.resize((height,width))
        image = np.array(image)

        dataset.append(image)
        labels.append(1)

  0%|          | 0/2726 [00:00<?, ?it/s]

100%|██████████| 2726/2726 [00:06<00:00, 414.07it/s]


In [6]:
# shuffling the dataset
indices = [i for i in range(len(labels))]

random.shuffle(indices)

dataset = [dataset[i] for i in indices]
labels = [labels[i] for i in indices]

#### Splitting the dataset

In [7]:
x_train, x_test, y_train,y_test = train_test_split(dataset, to_categorical(np.array(labels)), test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [8]:
print(len(x_train),len(y_train))
print(len(x_test),len(y_test))
print(len(x_val),len(y_val))

4361 4361
545 545
546 546


#### Building the model

In [9]:
Input_shape = (height,width,3)

inp = k.Input(shape=Input_shape)

conv1 = k.Conv2D(32, kernel_size = (3,3), activation='relu', padding="same")(inp)
pool1 = k.MaxPool2D(pool_size=(2,2))(conv1)
norm1 = k.BatchNormalization(axis = -1)(pool1)
drop1 = k.Dropout(rate = 0.2)(norm1)

conv2 = k.Conv2D(32, kernel_size = (3,3), activation='relu', padding="same")(drop1)
pool2 = k.MaxPool2D(pool_size=(2,2))(conv2)
norm2 = k.BatchNormalization(axis = -1)(pool2)
drop2 = k.Dropout(rate = 0.2)(norm2)

conv3 = k.Conv2D(32, kernel_size = (3,3), activation='relu', padding="same")(drop2)
pool3 = k.MaxPool2D(pool_size=(2,2))(conv3)
norm3 = k.BatchNormalization(axis = -1)(pool3)
drop3 = k.Dropout(rate = 0.2)(norm3)

flat = k.Flatten()(drop3)

hidden1 = k.Dense(512, activation = 'relu')(flat)
norm4 = k.BatchNormalization(axis = -1)(hidden1)
drop4 = k.Dropout(rate = 0.2)(norm4)

hidden2 = k.Dense(512, activation = 'relu')(drop4)
norm5 = k.BatchNormalization(axis = -1)(hidden2)
drop5 = k.Dropout(rate = 0.2)(norm5)

out = k.Dense(2, activation="sigmoid")(drop5)

In [10]:
model = keras.Model(inputs = inp, outputs = out)

model.compile(optimizer = Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 70, 70, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 70, 70, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 35, 35, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 35, 35, 32)        128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 35, 35, 32)        0         


                                                                 
 conv2d_1 (Conv2D)           (None, 35, 35, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 17, 17, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization_1 (Bat  (None, 17, 17, 32)        128       
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 17, 17, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 8, 8, 32)          0         
 g2D)                                                            
          

In [11]:
model.fit(np.array(x_train), y_train, batch_size=64, verbose=1, epochs=10, validation_data=(np.array(x_val),y_val))

Epoch 1/10


69/69 [==============================] - 13s 115ms/step - loss: 0.1999 - accuracy: 0.9321 - val_loss: 0.3019 - val_accuracy: 0.8846
Epoch 2/10
69/69 [==============================] - 7s 97ms/step - loss: 0.0832 - accuracy: 0.9725 - val_loss: 0.2961 - val_accuracy: 0.8810
Epoch 3/10
69/69 [==============================] - 7s 97ms/step - loss: 0.0579 - accuracy: 0.9823 - val_loss: 0.0854 - val_accuracy: 0.9835
Epoch 4/10
69/69 [==============================] - 7s 98ms/step - loss: 0.0439 - accuracy: 0.9846 - val_loss: 0.0489 - val_accuracy: 0.9872
Epoch 5/10
69/69 [==============================] - 7s 97ms/step - loss: 0.0408 - accuracy: 0.9890 - val_loss: 0.0829 - val_accuracy: 0.9762
Epoch 6/10
69/69 [==============================] - 7s 97ms/step - loss: 0.0470 - accuracy: 0.9842 - val_loss: 0.0502 - val_accuracy: 0.9872
Epoch 7/10
69/69 [==============================] - 7s 97ms/step - loss: 0.0302 - accuracy: 0.9904 - val_loss: 0.0248 - val_accuracy: 0.9908
Epoch 8/1

#### Model accuracy

In [16]:
print("Accuracy :",model.evaluate(np.array(x_test), y_test)[1]*100)

18/18 [==============================] - 0s 22ms/step - loss: 0.0172 - accuracy: 0.9945
Accuracy : 99.44953918457031


#### model save

In [12]:
with open('eye_state_classification_model.pkl','wb') as f:
    pickle.dump(model, f)

In [18]:
np.array([x_test[0]])

array([[[[120, 120, 120],
         [118, 118, 118],
         [118, 118, 118],
         ...,
         [109, 109, 109],
         [108, 108, 108],
         [107, 107, 107]],

        [[119, 119, 119],
         [117, 117, 117],
         [118, 118, 118],
         ...,
         [108, 108, 108],
         [108, 108, 108],
         [107, 107, 107]],

        [[119, 119, 119],
         [116, 116, 116],
         [119, 119, 119],
         ...,
         [107, 107, 107],
         [107, 107, 107],
         [107, 107, 107]],

        ...,

        [[110, 110, 110],
         [111, 111, 111],
         [112, 112, 112],
         ...,
         [105, 105, 105],
         [105, 105, 105],
         [105, 105, 105]],

        [[111, 111, 111],
         [111, 111, 111],
         [112, 112, 112],
         ...,
         [106, 106, 106],
         [106, 106, 106],
         [106, 106, 106]],

        [[111, 111, 111],
         [111, 111, 111],
         [112, 112, 112],
         ...,
         [106, 106, 106],
        

In [17]:
model.predict(np.array([x_test[0]]))

1/1 [==============================] - 0s 53ms/step


array([[6.926796e-04, 9.982860e-01]], dtype=float32)